In [109]:
import numpy as np
from skimage import io
from skimage import util
from skimage.util import img_as_float
from skimage import color
from skimage import feature
from skimage.transform import resize, rescale
import os
from random import randint, randrange
from modules.utils import area_rate, get_shape_stats
from sklearn.svm import LinearSVC

In [110]:
# Chargement des images
images = []
path = "data/train/"
for index, img_file in enumerate(os.listdir(path)):
    images.append(img_as_float(io.imread(path + img_file)))

In [111]:
# Chargement des labels positifs
label_file = np.loadtxt("data/label.txt")
label_class = np.ones((len(label_file),1))
positive_labels = np.concatenate((label_file,label_class), axis=1)
POS_SIZE = len(positive_labels)

In [128]:
# Génération des labels négatifs

NEG_SIZE = 1000
MIN_H = 40
MIN_L = 40

n_images = len(images)

negative_labels = np.zeros((NEG_SIZE,6))

i = 0
while i < NEG_SIZE:
    img_index = randrange(n_images)
    img_id = img_index + 1
    
    img = images[img_index]
    
    
    img_h, img_l = img.shape[:2]
    
    h = randint(MIN_H, int(img_h/2))
    l = randint(MIN_L, int(img_l/2))
    
    x = randint(0, img_h - h)
    y = randint(0, img_l - l)
    
    # Check if it doesn't overlap with true faces
    img_labels = labels[labels[:,0] == img_id]
    if all([ area_rate([x,y,h,l], label[1:5]) < 1/3 for label in labels ]):
        negative_labels[i] = [ img_id,x,y,h,l, -1 ]
        i += 1

C:\Users\Josselin\Documents\UTC maggle\GI04\SY32\Projet\modules\utils.py:48: RuntimeWarning: divide by zero encountered in double_scalars
  return a_in / (area(box_1) + area(box_2) - a_in)


In [129]:
# Assemblage des labels
labels = np.concatenate((positive_labels, negative_labels))

In [131]:
# Récupération des images et création des vecteurs descripts avec HOG
X = np.zeros((POS_SIZE + NEG_SIZE, 729))

for index, label in enumerate(labels):
    img_id,x,y,h,l,_ = labels[index]
    img = images[int(img_id - 1)][int(x):int(x+h), int(y):int(y+l)]
    resized_img = resize(img, (40,40), mode='constant', anti_aliasing=True)
#     print(feature.hog(resize(img, (40,40), mode='constant', anti_aliasing=True)).shape)
    X[i] = feature.hog(resized_img, block_norm='L1')

y = labels[:,5]

In [132]:
# Entraînement
clf = LinearSVC()
clf.fit(X,y)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [143]:
# Deuxième phase d'apprentissage

test_labels = []
step_h = 40
step_l = 40
scales = [0.25, 0.5, 0.75, 1.0]

img_to_predict = np.zeros((1, 729))
window_labels = []

for index, image in enumerate(images):
    
#     if index > 10 :
#         break;
    
    img_id = index + 1
    
    for scale in scales:
        
        rescaled_img = rescale(image, scale, mode='constant', anti_aliasing=True, multichannel=False)
        
        rescaled_img_h, rescaled_img_l = rescaled_img.shape[:2]
        
        for x in range(0, rescaled_img_h, step_h):
            
            for y in range(0, rescaled_img_l, step_l):

                if x + step_h < rescaled_img_h and y + step_l < rescaled_img_l:
                
                    
                    img_to_predict[0] = feature.hog(rescaled_img[x:x+step_h, y:y+step_l], block_norm='L1')
                    
                    prediction = clf.decision_function(img_to_predict)
                    window_labels.append(prediction[0])
#                     window_labels.append([img_id, x/scale, y/scale, step_h/scale, step_l/scale,prediction[0]])


In [148]:
test = np.array(window_labels)
# print(np.mean(test[:,5]))
# print(np.min(test[:,5]))
# print(np.max(test[:,5]))
print(np.mean(test))
print(np.min(test))
print(np.max(test))
print(test[3:40])
print(len(np.where(test > 0.5)))

0.2649066297696365
0.1241847211351379
0.5635186967346619
[0.27741612 0.26378529 0.34236541 0.24013297 0.20732177 0.32317251
 0.27503941 0.27239812 0.32178347 0.21151522 0.31045486 0.20904856
 0.24768833 0.3533247  0.23143929 0.34445054 0.31872525 0.25490304
 0.25066015 0.24385787 0.21263269 0.19658365 0.27720667 0.28090803
 0.26141651 0.31437757 0.26724653 0.30855903 0.34293543 0.25605754
 0.24647031 0.20630392 0.26581011 0.27511214 0.26740339 0.31609327
 0.27046851]
1


In [147]:
img_id,x,y,h,l, _ = labels[0]
test = resize(images[0][int(x):int(x+h), int(y):int(y+l)], (40,40))
im = np.zeros((1,729))
im[0] = feature.hog(test, block_norm='L1')
pred = clf.predict(im)
print(pred)

[1.]
